In [1]:
%%capture
%pip install --upgrade iblm

In [2]:
import sys
sys.path.append("../..")
import pandas as pd

from iblm import ibl

from src.data import pseudodata
from sklearn.datasets import make_classification


In [63]:
num_train_data = 50 # 50〜300
num_test_data = 1000
sample = num_train_data + num_test_data
n_informative = 4
n_redundant = 0
n_features = n_informative + n_redundant
weights = [0.5, 0.5]
flip_y=0.1
seed = 3655  # 3655,3656,3657

# testデータの個数を揃えるためにtrain_test_splitを使っていない
x, y = make_classification(
    n_samples = sample,  # データ数
    n_features = n_features,  # 特徴量の数
    n_informative = n_informative,  # ラベル予測に意味のある特徴量の数
    n_redundant = n_redundant,  # 冗長な特徴量
    weights = weights,  # [0,1]の割合
    flip_y = flip_y, # 逆のラベルに反転する割合
    random_state = seed
    )
x = pd.DataFrame(x)

x_train = x[0:num_train_data]
x_test = x[num_train_data:]

y_train = y[:num_train_data]
y_test = y[num_train_data:]

In [64]:
iblm = ibl.IBLModel(api_type="openai", model_name="gpt-4-0125-preview", objective="binary")

In [68]:
model = iblm.fit(x_train, y_train)

2024-02-11 21:18:48,984 [iblm.ibl][INFO] (ibl:ibl.py:fit:169)


In [69]:
print(model)

#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Custom logic based on data analysis
        # Feature engineering and thresholding based on observed patterns
        feature_1_weight = 0.25
        feature_2_weight = 0.15
        feature_3_weight = 0.3
        feature_4_weight = 0.3
        threshold = 0.5

        # Calculating a simple weighted sum of features
        weighted_sum = (row[0] * feature_1_weight + 
                        row[1] * feature_2_weight + 
                        row[2] * feature_3_weight + 
                        row[3] * feature_4_weight)

        # Normalizing the weighted sum to get a probability-like value
        normalized_weighted_sum = (weighted_sum + 4) / 8  # Assuming -4 to 4 as the range of weighted_sum

        # Applying threshold to decide on the probability
        if normalized_weighted_sum > threshold:
            y = normalized_weighted_sum  # Higher probabi

In [70]:
y_pred = iblm.predict(x_test)

In [71]:
iblm.evaluate(y_test, y_pred)

{'roc_auc': 0.3819261926192619,
 'pr_auc': 0.43230467749418755,
 'accuracy': 0.505,
 'recall': 1.0,
 'precision': 0.505,
 'f1_score': 0.6710963455149501}